<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/BPairsEigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook compute all eigentensor pairs by Rayleigh quotient iteration for a generalized Tensor eigenvalue problem
* Eigenpairs  
$$\cT(I, X^{[m-1]}) =\lambda \cB(I, X^{[d-1]}) $$
The square bracket means the number of times $X$ is repeated.
* $\cT$ is a $(m, n)$ tensor, $\cB$ is a $(d, n)$ tensor. Both with coefficients in $\R$. However $X$ could be in $\C$.

* Use generalized Rayleigh quotient method. The purpose is to show it is easy to pick different Rayleigh quotient and retractions to solve some well-known problems.
* Two approaches, using different choices of Rayleigh quotient and retraction.
* For the real case, $X\in \R^n$, consider the constraints $\cB(X^{[d]}) = 1$. This is not the most general case, we could have eigenpairs with $\cB(X^{[d]}) = -1$ or $\cB(X^{[d]}) = 0$, but we consider this case for simplicity. The left inverse is $ X^T$, the Rayleigh quotient is 
$$\lambda = \cT(X^{[m]}).$$

* For the complex case, $X\in \C^n$, consider the unitary constraint $X^*X = 1$. In this case we assume $m\neq d$. The left inverse is $\omega\mapsto \Re(\frac{1}{|\cB(I, X^{[d-1]})|^2}\cB(I, X^{[d-1]})^*\omega)$. The Rayleigh quotient is
$$\frac{1}{|\cB(I, X^{[d-1]})|^2}\Re(\cB(I, X^{[d-1]})^*\cT(I, X^{[m-1]}))$$

The eigenvalue count is 
$$\frac{(m-1)^n - (d-1)^n}{m - d} = \sum_{i=0}^{m-1}(m-1)^i(d-1)^{n-1-i}
$$
* In contrast to the workbook ZPairsEigentensor, since we want to test the eigen pair count, we do not stop after the count is reached, but keep running for sometime, showing we do not get more pairs after the count is reached.


* Some cells may be hidden. Please download and open in colab, then expand the hidden cells
First, clone the project from github


In [1]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 249 (delta 130), reused 244 (delta 125), pack-reused 0
Receiving objects: 100% (249/249), 14.73 MiB | 12.82 MiB/s, done.
Resolving deltas: 100% (130/130), done.


Importing main functions to be used later - but the code to find all complex eigen pairs is in the next block.

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
import time
import rayleigh_newton.core.utils as utils

import sys
import scipy.linalg
from numpy import concatenate, tensordot, eye, zeros, zeros_like,\
    power, sqrt, exp, pi

from numpy.linalg import solve, inv, norm

if sys.version_info[0] < 3:
    class SimpleNamespace:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)

            def __repr__(self):
                keys = sorted(self.__dict__)
                items = ("{}={!r}".format(k, self.__dict__[k]) for k in keys)
                return "{}({})".format(type(self).__name__, ", ".join(items))

            def __eq__(self, other):
                return self.__dict__ == other.__dict__
else:
    from types import SimpleNamespace


def symmetric_tv_mode_product(T, x, modes) -> np.ndarray:
    v = T
    for i in range(modes):
        v = tensordot(v, x, axes=1)
    return v



First, the real case with constraint $B(X^{[d]}) = \epsilon$.
The tangent space consisting of $\eta$ with $B(\eta, X^{[d-1]}) = 0$. For this, we create a class Bfeasible to model the feasible set. We include a retraction, sending a pair $(X, \eta)$ to a point $\gamma(X+\eta)$ on the feasible set with $\gamma > 0$.

The Rayleigh quotient is $\lambda= R(X)= \epsilon T(X^{[m]})$.



In [3]:
def rroots(v, d):
  if (v < 0) and (d % 2 == 0):
      return np.nan
  return np.sign(v)*np.abs(v)**(1/d)

class Bfeasible(object):
  def __init__(self, B):
    self.B = B    
    self.n = B.shape[0]
    self.d = len(B.shape)

  def rand(self):
    while True:
        x = np.random.randn(self.n)
        vv = symmetric_tv_mode_product(self.B, x, modes=self.d)
        if vv > 0:
            break
    return x / rroots(vv, self.d)

  def proj_tan(self, x, omg):
      V = symmetric_tv_mode_product(self.B, x, modes=self.d-1)
      return omg - x*np.sum(V*omg)

  def Pi(self, x, omg):
      V = symmetric_tv_mode_product(self.B, x, modes=self.d-1)
      return omg - V*np.sum(x*omg)

  def randvec(self, x):
      return self.proj_tan(x, np.random.randn(self.n))

  def tensor_rtr(self, x, eta):
    v = x+eta
    V = symmetric_tv_mode_product(self.B, v, modes=self.d)
    return v / rroots(V, self.d)

  def D_tensor_rtr(self, x, xi, eta):
    v = x+xi    
    VB1 = symmetric_tv_mode_product(self.B, v, modes=self.d-1)
    VB = np.sum(VB1*v)
    return eta / rroots(VB, self.d) - v/VB/rroots(VB, self.d)*np.sum(VB1*eta)

  def H_tensor_rtr(self, x, eta):
    # Hessian of the retraction
    V2 = symmetric_tv_mode_product(self.B, x, modes=self.d-2)
    return -(self.d-1)*x*np.sum(eta*(V2@eta))    


Test this retraction

In [4]:

n = 5
m = 4
d = 3

T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)

Bf = Bfeasible(B)
x = Bf.rand()
eta = Bf.randvec(x)
# symmetric_tv_mode_product(B, x, modes=d)
display(np.sum(symmetric_tv_mode_product(B, x, modes=d-1)*eta))
x1 = Bf.tensor_rtr(x, eta)
display(symmetric_tv_mode_product(B, x, modes=d))
print(symmetric_tv_mode_product(B, x1, modes=d))

# test D_tensor_rtr:
dlt = 1e-7
eta1 = Bf.randvec(x)
print((Bf.tensor_rtr(x, eta +dlt*eta1) - Bf.tensor_rtr(x, eta))/dlt - Bf.D_tensor_rtr(x, eta, eta1))
print((Bf.D_tensor_rtr(x, dlt*eta, eta) - Bf.D_tensor_rtr(x, 0, eta))/dlt - Bf.H_tensor_rtr(x, eta))


5.551115123125783e-16

array(1.)

0.9999999999999997
[-3.11452258e-08  1.11908164e-07 -1.23297551e-07 -6.94111459e-08
  1.21814603e-07]
[ 8.53645020e-07 -1.53949737e-06  1.15902540e-06  3.26672165e-08
 -7.76687919e-07]


# The Schur form iteration for the B tensor eigenpair

In [5]:
def schur_form_B_tensor_rayleigh_chebyshev(
        T, Bf, max_itr, delta, x_init=None, do_chebyshev=True):
    """Schur form rayleigh chebyshev 
    T and x are complex. Constraint is B(x^{[d]}) = 1
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    B = Bf.B
    d = Bf.d
    R = 1    
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = Bf.rand()        

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    if do_chebyshev:
        T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
        T_x_m_2 = tensordot(T_x_m_3, x_k, axes=1)
        B_x_m_3 =  symmetric_tv_mode_product(Bf.B, x_k, d-3)
        B_x_m_2 =  tensordot(B_x_m_3, x_k, axes=1)
    else:
        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        B_x_m_2 = symmetric_tv_mode_product(Bf.B, x_k, d-2)

    T_x_m_1 = T_x_m_2 @ x_k
    B_x_m_1 = B_x_m_2 @ x_k

    lbd = np.sum(x_k.T * T_x_m_1)
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [B_x_m_1.reshape(-1, 1), T_x_m_1.reshape(-1, 1) -lbd*B_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2 - (d-1)*B_x_m_2*lbd
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((B_x_m_1 * lhs[:, 1])) /
            np.sum((B_x_m_1 * lhs[:, 0]))) - lhs[:, 1]
        if do_chebyshev and (np.linalg.norm(y) < 30e-2):
            D_R_eta = m * np.sum(T_x_m_1@y)
            L_x_lbd = -(d-1)*(B_x_m_2@y) * D_R_eta            
            L_x_x = (m-1) * (m-2) * np.tensordot(T_x_m_3, y, axes=1) @ y -\
              (d-1)*(d-2)*np.tensordot(B_x_m_3, y, axes=1) @ y

            L_x_H_rtr = ((m-1)*T_x_m_2 - (d-1)*B_x_m_2*lbd)@Bf.H_tensor_rtr(x_k, y)
            # we only need L_x_x but put the other ones for completeness
            T_a = np.linalg.solve(H, 2*L_x_lbd + L_x_x + L_x_H_rtr)
            # T_a = np.linalg.solve(H, L_x_x)
            T_adj = T_a - lhs[:, 0] *\
                np.sum(B_x_m_1 * T_a) / np.sum(B_x_m_1 * lhs[:, 0])
            # print(np.sum(B_x_m_1*T_adj))
            x_k_n = Bf.tensor_rtr(x_k, y - 0.5*T_adj)
        else:
            x_k_n = Bf.tensor_rtr(x_k, y)

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n
        # import pdb
        # pdb.set_trace()
        if do_chebyshev:
            T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
            T_x_m_2 = np.tensordot(T_x_m_3, x_k, axes=1)
            
            B_x_m_3 = symmetric_tv_mode_product(Bf.B, x_k, d-3)
            B_x_m_2 = np.tensordot(B_x_m_3, x_k, axes=1)
            B_x_m_1 = B_x_m_2 @ x_k
        else:
            T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
            B_x_m_2 = symmetric_tv_mode_product(Bf.B, x_k, d-2)
        T_x_m_1 = T_x_m_2 @ x_k            
        B_x_m_1 = B_x_m_2 @ x_k

        lbd = np.sum(x_k * T_x_m_1)
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * symmetric_tv_mode_product(
        Bf.B, x, d-1))
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err

def Rayp(T, Bf, x, eta):
    # FF = F(x)
    VA2 = symmetric_tv_mode_product(T, x, m-2)
    VA1 = VA2@x
    return np.sum(eta*VA1) + (m-1)*np.sum(x*(VA2@eta))
    
    # return np.sum(eta*FF[0]) + np.sum(x*(FF[2]@eta+FF[5]@eta*FF[1]))

def GL(x, eta):
    VA = eval(A, x)
    VB = eval(B, x)
    lbd = np.sum(x*VA[1])        
    rp = np.sum(eta*VA[1]) + (m-1)*np.sum(x*(VA[2]@eta))
    # print(rp)

    if m == 2:
        G = - (d-1)*(d-2)*lbd*np.tensordot(np.tensordot(VB[3], eta, axes=1), eta, axes=1) \
            + 2*(d-1)*VB[2]@eta*rp \
            + ((m-1)*VA[2] - (d-1)*VB[2]*lbd)@r2(x, eta)
        
        return G
    G = (m-1)*(m-2)*np.tensordot(np.tensordot(VA[3], eta, axes=1), eta, axes=1) \
        - (d-1)*(d-2)*lbd*np.tensordot(np.tensordot(VB[3], eta, axes=1), eta, axes=1) \
        + 2*(d-1)*VB[2]@eta*rp \
        + ((m-1)*VA[2] - (d-1)*VB[2]*lbd)@r2(x, eta)
    return G

def Lx(T, Bf, x, lbd):
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    d = Bf.d
    VT2 = symmetric_tv_mode_product(T, x, m-2)
    VB2 = symmetric_tv_mode_product(Bf.B, x, d-2)
    return (m-1)*VT2 - (d-1)*VB2*lbd

def Lxx(T, Bf, x, lbd):
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    d = Bf.d
    VT3 = symmetric_tv_mode_product(T, x, m-3)
    VB3 = symmetric_tv_mode_product(Bf.B, x, d-3)  
    return (m-2)*(m-1)*VT3 - (d-2)*(d-1)*VB3*lbd


Test run for the $B$ tensor pair. Showing at a solution various components of the Chebyshev term is zero.

The only non zero term is $L_{XX}$

In [6]:
import numpy as np
import numpy.linalg as la

n = 5
m = 4
d = 3
# np.random.seed(0)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)
max_itr = 200
Bf = Bfeasible(B)
ret = schur_form_B_tensor_rayleigh_chebyshev(
        T, Bf, max_itr, delta=1e-8, x_init=None, do_chebyshev=False)
print('doing RQI')
print(ret)
ret = schur_form_B_tensor_rayleigh_chebyshev(
        T, Bf, max_itr, delta=1e-8, x_init=None, do_chebyshev=True)
print('doing Rayleigh-Chebyshev')
print(ret)

x = ret[0]

lbd = symmetric_tv_mode_product(T, x, m)
# show various components are zeros

print('Derivative of Rayleigh quotient at x_* = %.5f' % Rayp(T, Bf, x, Bf.randvec(x)))
print('Projection of Lx Hessian of retraction =%s' % str(Bf.Pi(x, Lx(T, Bf, x, lbd)@Bf.H_tensor_rtr(x, eta))))
print('Projection of L_lambda=%s' % str(Bf.Pi(x, symmetric_tv_mode_product(Bf.B, x, Bf.d-1))))


doing RQI
(array([-0.25172172,  0.15594528, -0.5344076 , -0.71947059,  1.48122345]), 3.216611422092537, 7, True, 1.191874962812805e-15)
doing Rayleigh-Chebyshev
(array([ 0.45695372,  0.27863962,  0.25540008, -0.08373419,  0.30859434]), 1.0735566708660087, 7, True, 3.1401849173675503e-16)
Derivative of Rayleigh quotient at x_* = -0.00000
Projection of Lx Hessian of retraction =[-6.66133815e-16  2.22044605e-16 -2.22044605e-16 -2.22044605e-16
 -2.22044605e-16]
Projection of L_lambda=[-2.22044605e-16 -1.11022302e-16 -2.22044605e-16 -1.11022302e-16
 -1.11022302e-16]


THis confirms the analysis in the paper.

# The Unitary $\cB$ pairs.
The constraint is $X^*X = 1$. They Rayleigh quotient is
$\frac{1}{|\cB(I, X^{[d-1]})|^2}\Re(\cB(I, X^{[d-1]})^*\cT(I, X^{[m-1]}))$, where $\Re$ is the real part.

We 


In [7]:
def schur_form_B_tensor_rayleigh_orthogonal(
        T, B, max_itr, delta, x_init=None):
    """Schur form rayleigh chebyshev unitary
    T and x are complex. Constraint is x^H x = 1
    lbd is real
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k

    B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
    B_x_m_1 = B_x_m_2 @ x_k

    lbd = (B_x_m_1.T @ T_x_m_1).real/norm(B_x_m_1)**2
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [B_x_m_1.reshape(-1, 1), T_x_m_1.reshape(-1, 1)-lbd*B_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*(d-1)*B_x_m_2
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((x_k * lhs[:, 1])) /
            np.sum((x_k * lhs[:, 0]))) - lhs[:, 1]
        x_k_n = (x_k + y) / norm(x_k + y)

        # x_k_n = (x_k + y)/(np.linalg.norm(x_k + y))

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n

        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k
        B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
        B_x_m_1 = B_x_m_2 @ x_k

        lbd = (B_x_m_1.T @ T_x_m_1).real/norm(B_x_m_1)**2
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * symmetric_tv_mode_product(
        B, x, d-1))
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err

def schur_form_B_tensor_rayleigh_unitary(
        T, B, max_itr, delta, x_init=None):
    """Schur form rayleigh chebyshev unitary
    T and x are complex. Constraint is x^H x = 1
    lbd is real
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k

    B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
    B_x_m_1 = B_x_m_2 @ x_k

    lbd = (B_x_m_1.conjugate().T @ T_x_m_1).real/norm(B_x_m_1)**2
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [B_x_m_1.reshape(-1, 1), T_x_m_1.reshape(-1, 1)-lbd*B_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*(d-1)*B_x_m_2
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((x_k.conjugate() * lhs[:, 1]).real) /
            np.sum((x_k.conjugate() * lhs[:, 0]).real)) - lhs[:, 1]
        x_k_n = (x_k + y) / norm(x_k + y)

        # x_k_n = (x_k + y)/(np.linalg.norm(x_k + y))

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n

        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k
        B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
        B_x_m_1 = B_x_m_2 @ x_k

        lbd = (B_x_m_1.conjugate().T @ T_x_m_1).real/norm(B_x_m_1)**2
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * symmetric_tv_mode_product(
        B, x, d-1))
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err

def complex_eigen_cnt(n, m, d):
    if m == d:
        return n*power(m-1, n-1)
    return (power(m-1, n)-power(d-1, n)) // (m-d)


def find_eig_cnt(all_eig):
    first_nan = np.where(np.isnan(all_eig.x))[0]
    if first_nan.shape[0] == 0:
        return None
    else:
        return first_nan[0]

    
def normalize_real_positive(lbd, x, m, d, tol):
    """ First try to make it to a real pair
    if not possible. If not then make lambda real
    return is_self_conj, is_real, new_lbd, new_x
    """
    u = sqrt((x @ x).conjugate())
    new_x = x * u
    is_real = norm(new_x.imag) < m*tol
    if is_real:
        # if zero, just insert
        if np.abs(lbd) < tol:
          return True, True, lbd, new_x
        # try to flip. if u **(m-d) > 0 use it:
        lbd_factor = u**(m-d)
        if np.abs(lbd_factor*lbd_factor - 1) < tol:
            lbd_factor = lbd_factor.real
            if lbd * lbd_factor > 0:
                return True, True, lbd * lbd_factor, new_x
            elif (m - d) % 2 == 1:
                return True, True, -lbd * lbd_factor, -new_x
            else:
                return True, True, lbd * lbd_factor, new_x

    is_self_conj = np.abs((x@x).conjugate()**(m-d)-1) < tol
    if lbd < 0:
        return is_self_conj, False, -lbd, x * exp(pi/(m-d)*1j)
    else:
        return is_self_conj, False, lbd, x


def _insert_eigen(all_eig, x, lbd, eig_cnt, m, d, tol, disc):
    """
    force eigen values to be positive if possible
    if x is not similar to a vector in all_eig.x
    then:
       insert pair x, conj(x) if x is not self conjugate
       otherwise insert x
    all_eig has a structure: lbd, x, is_self_conj, is_real
    """
    is_self_conj, is_real, norm_lbd, norm_x = normalize_real_positive(
        lbd, x, m, d, tol)

    if is_self_conj:
        good_x = [norm_x]
    else:
        good_x = [norm_x, norm_x.conjugate()]
    nct = 0
    for xx in good_x:
      factors = all_eig.x[:eig_cnt+nct, :] @ xx.conjugate()
      fidx = np.where(np.abs(factors ** (m-d) - 1) < disc)[0]
      if fidx.shape[0] == 0:
        all_eig.lbd[eig_cnt+nct] = norm_lbd
        all_eig.x[eig_cnt+nct] = xx
        all_eig.is_self_conj[eig_cnt+nct] = is_self_conj
        all_eig.is_real[eig_cnt+nct] = is_real
        nct += 1
    return eig_cnt + nct

def old_insert_eigen(all_eig, x, lbd, eig_cnt, m, d, tol):
    """
    force eigen values to be positive if possible
    if x is not similar to a vector in all_eig.x
    then:
       insert pair x, conj(x) if x is not self conjugate
       otherwise insert x
    all_eig has a structure: lbd, x, is_self_conj, is_real
    """
    is_self_conj, is_real, norm_lbd, norm_x = normalize_real_positive(
        lbd, x, m, d, tol)

    if is_self_conj:
        good_x = [norm_x]
    else:
        good_x = [norm_x, norm_x.conjugate()]

    factors = all_eig.x[:eig_cnt, :] @ norm_x.conjugate()
    fidx = np.where(np.abs(factors ** (m-d)-1) < tol)[0]
    if fidx.shape[0]:
        all_diffs = all_eig.x[:eig_cnt, :][fidx, :] -\
                    factors[fidx][:, None] * norm_x[None, :]
        if np.where(np.sum(np.abs(all_diffs), axis=1) < tol * x.shape[0])[0].shape[0]:
            return eig_cnt

    for j in range(len(good_x)):
        all_eig.lbd[eig_cnt+j] = norm_lbd
        all_eig.x[eig_cnt+j] = good_x[j]
        all_eig.is_self_conj[eig_cnt+j] = is_self_conj
        all_eig.is_real[eig_cnt+j] = is_real

    return eig_cnt + len(good_x)


def find_all_unitary_eigenpair(
        all_eig, eig_cnt, A, B, max_itr, max_test=int(1e5), tol=1e-6, disc=1e-3):
    """ output is the table of results
     2n*+2 columns: lbd, is self conjugate, x_real, x_imag
    This is the raw version, since the output vector x
    is not yet normalized to be real when possible
    """
    n = A.shape[0]
    m = len(A.shape)
    d = len(B.shape)
    if m == d:
      print("cannot deal with m=%d = n=%d" % (m, d))
      return None, None
    # n_eig = complex_eigen_cnt(n, m)
    n_eig_proj = complex_eigen_cnt(n, m, d)
    n_eig = all_eig.lbd.shape[0]
    if all_eig is None:
        all_eig = SimpleNamespace(
            lbd=np.full((n_eig), np.nan, dtype=float),
            x=np.full((n_eig, n), np.nan, dtype=complex),
            is_self_conj=zeros((n_eig), dtype=bool),
            is_real=zeros((n_eig), dtype=bool))
        eig_cnt = 0
    elif eig_cnt is None:
        eig_cnt = find_eig_cnt(all_eig)
        if eig_cnt is None:
            return all_eig

    for jj in range(max_test):
        x0r = np.random.randn(2*n)
        x0r /= norm(x0r)
        x0 = x0r[:n] + x0r[n:] * 1.j
        # if there are odd numbers left,
        # try to find a real root
        draw = np.random.uniform(0, 1, 1)
        # 50% try real root
        if True and (draw < .5) and ((n_eig_proj - eig_cnt) % 2 == 1):
            try:
                x_r, lbd, ctr, converge, err = schur_form_B_tensor_rayleigh_orthogonal(
                    A, B, max_itr, tol, x_init=x0.real)
                x = x_r + 1j * zeros((x_r.shape[0]))
            except Exception as e:
                print(e)
                continue
        else:
            try:
                x, lbd, ctr, converge, err =\
                    schur_form_B_tensor_rayleigh_unitary(
                        A, B, max_itr, tol, x_init=x0)
            except Exception as e:
                print(e)
                continue
        old_eig = eig_cnt
        if converge and (err < tol):
            eig_cnt = _insert_eigen(all_eig, x, lbd, eig_cnt, m, d, tol, disc)
        if eig_cnt == n_eig:
            break
        # elif (eig_cnt > old_eig) and (eig_cnt % 10 == 0):
        elif (eig_cnt > old_eig) and True:
            print('Found %d eigenpairs' % eig_cnt)
    return SimpleNamespace(
            lbd=all_eig.lbd[:eig_cnt],
            x=all_eig.x[:eig_cnt, :],
            is_self_conj=all_eig.is_self_conj[:eig_cnt],
            is_real=all_eig.is_real[:eig_cnt]), eig_cnt
    
def save_out(T, B, all_eig, save_file):
    np.savez_compressed('%s_%d_%d_%d.npz' % (
        save_file, n, m, d), T=T, B=B, lbd=all_eig.lbd,
        x=all_eig.x, is_real=all_eig.is_real,
        is_self_conj=all_eig.is_self_conj)



Time running a random (4, 6, 3) tensor

In [8]:
n = 3
m = 4
d = 6
np.random.seed(0)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)

ret = schur_form_B_tensor_rayleigh_unitary(
    T, B, max_itr=200, delta=1e-8, x_init=None)
print(ret)

ret = schur_form_B_tensor_rayleigh_orthogonal(
    T, B, max_itr=200, delta=1e-8, x_init=None)
print(ret)


(array([-0.23855975, -0.43257377, -0.86946488]), 0.5716317436131156, 9, True, 1.0877919644084146e-15)
(array([ 0.78461508, -0.54484555,  0.29584202]), 0.17592464459560272, 12, True, 9.205483015737869e-17)


Get all pairs of a $(m=3, d=5, n=3)$ tensor.
We test with max_test $= 10^4$ intial points.  For a more difficult tensor we increase max_test to $10^5$ or $10^6$.

The expected eigen pair count is $\frac{(m-1)^n-(d-1)^n}{m-d}=28$. We did extensive test - and the count never exceed this count.

In [9]:
n = 3
m = 3
d = 5
np.random.seed(0)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)
n_eig = 1200
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=float),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=2e-3)  
print(jcnt)

Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
28


Some codes to beautify output

In [10]:
def check_eig(T, B, all_eig):
  m = len(T.shape)
  d = len(B.shape)
  max_run = all_eig.lbd.shape[0]
  # max_run = 76
  diff0 = np.empty(max_run)
  diff1 = np.empty(max_run)  
  for i in range(max_run):
    Tnew = symmetric_tv_mode_product(T, all_eig.x[i, :], m-1)
    Bnew = symmetric_tv_mode_product(B, all_eig.x[i, :], d-1)
    lbdnew = np.sum(Bnew.conjugate()*Tnew)/norm(Bnew)**2
    diff0[i] = lbdnew - all_eig.lbd[i]
    diff1[i] = norm(Tnew - lbdnew*Bnew)
  print('check lbd %f' % np.max(np.abs(diff0)))
  print('check equation %f' % np.max(np.abs(diff1)))

  diff3 = np.empty(max_run-1)
  for i in range(max_run-1):
      factors = np.sum(all_eig.x[i, :].conj()*all_eig.x[i+1, :])
      diff3[i] = np.abs(np.abs(factors)-1)
  # print(np.argsort(diff3))
  print("check uniqueness")
  print(np.sort(diff3[np.where(diff3< 1e-1)]))

def display_one(comb, i):
  return (comb.x)[i, :], (comb.lbd)[i], (comb.is_real)[i], (comb.is_self_conj)[i]

def display_all_real(comb)  :
  return pd.DataFrame(
      index=np.where(comb.is_real)[0],
       data=np.concatenate([comb.lbd[np.where(comb.is_real)[0]].real[:, None], comb.x[np.where(comb.is_real)[0], :].real], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])

def display_all_complex(comb)  :
  return pd.DataFrame(
      index=np.where(comb.is_real==False)[0],
       data=np.concatenate([comb.lbd[np.where(comb.is_real==False)[0]].real[:, None], comb.x[np.where(comb.is_real==False)[0], :]], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])
    


In [11]:
check_eig(T, B, all_eig)
display_all_real(all_eig)


check lbd 0.000000
check equation 0.000000
check uniqueness
[0.02451759 0.0362202  0.09855531]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


,lbd,0,1,2
22,0.529238,-0.752803,-0.658071,0.015149
23,1.267230,0.102573,0.038814,0.993968


Another set of tests. We wait for a long time but get only 65 pairs, confirming the counting formula experimentailly

In [12]:
n = 4
m = 4
d = 3

np.random.seed(1)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)
n_eig0 = complex_eigen_cnt(n, m, d)
print("expected pairs %d" % n_eig0)
n_eig = 2*n_eig0
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=float),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(1.3e5), tol=1e-10)  
check_eig(T, B, all_eig)
display_all_real(all_eig)


expected pairs 65
Found 1 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 7 eigenpairs
Found 9 eigenpairs
Found 11 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 23 eigenpairs
Found 25 eigenpairs
Found 26 eigenpairs
Found 27 eigenpairs
Found 29 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 50 eigenpairs
Found 51 eigenpairs
Found 53 eigenpairs
Found 55 eigenpairs
Found 57 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 61 eigenpairs
Found 63 eigenpairs
Found 65 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.07073044 0.07113721 0.09457314]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


,lbd,0,1,2,3
0,0.682627,0.156708,0.928920,-0.290651,0.167547
17,1.649466,0.389731,-0.719881,-0.235711,0.523757
22,0.073690,0.008008,0.056804,-0.985744,0.158169
25,2.988166,0.002630,-0.387164,0.669546,0.633882
26,1.044213,-0.004337,0.907327,-0.336312,0.252254
49,0.644903,-0.307130,-0.066154,-0.312177,0.896571
50,1.790138,0.310316,-0.068910,0.260327,0.911694
57,2.057564,-0.239861,-0.654157,0.302534,0.650398
60,15.175945,0.648579,-0.522856,0.526916,-0.168305


In [13]:
n = 3
m = 4
d = 6
np.random.seed(2)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)

n_eig0 = complex_eigen_cnt(n, m, d)
n_eig = n_eig0*2
print("expected pairs %d" % n_eig0)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=float),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-10)  
check_eig(T, B, all_eig)
display_all_real(all_eig)  

expected pairs 49
Found 2 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 7 eigenpairs
Found 9 eigenpairs
Found 11 eigenpairs
Found 13 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 42 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.00093952 0.00266693 0.0097714  0.03054183 0.03105128 0.09021551]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


,lbd,0,1,2
2,0.621483,-0.054585,-0.979281,-0.195012
13,0.458756,0.766144,0.263520,0.586158
14,-0.067611,0.577293,-0.503623,-0.642726
19,0.409769,-0.605864,0.789072,0.101462
20,0.790021,0.200940,-0.941194,-0.271618
23,0.415954,0.779421,-0.561116,0.278660
30,0.620058,-0.952090,0.077905,-0.295730
41,-2.095027,0.464456,0.631381,-0.620998
42,-2.094362,-0.429172,-0.656552,0.620283


In [14]:
n = 3
m = 7
d = 5

np.random.seed(5)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, d)

n_eig0 = complex_eigen_cnt(n, m, d)
n_eig = 2*n_eig0
print("expected pairs %d" % n_eig0)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=float),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(8e3), tol=1e-10)  
check_eig(T, B, all_eig)
display_all_real(all_eig)

expected pairs 76
Found 1 eigenpairs
Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 38 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 47 eigenpairs
Found 48 eigenpairs
Found 50 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 59 eigenpairs
Found 60 eigenpairs
Found 62 eigenpairs
Found 64 eigenpairs
Found 66 eigenpairs
Found 68 eigenpairs
Found 70 eigenpairs
Found 72 eigenpairs
Found 73 eigenpairs
Found 74 eigenpairs
Found 76 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.02767111 0.03110071 0.03360667 0.04897567 0.05978839 0.05990102
 0.08644542 0.090468

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


,lbd,0,1,2
0,2.227287,0.528802,0.226161,0.818058
1,1.774110,0.429087,0.002529,0.903260
46,-0.507552,0.547157,-0.750971,-0.369679
47,2.254049,-0.433819,-0.321104,-0.841839
58,-0.488928,-0.371719,0.857257,0.356280
59,2.271352,-0.772801,-0.263005,-0.577588
72,-1.941106,0.364506,-0.930997,-0.019468
73,-44.446841,-0.850094,-0.218712,0.479067


In the previous example, the number of pairs found may be less than the total number of pairs, because of the random nature of the algorithm. A version of the Kantorovich theorem could be used to avoid picking initial points in area already tried.
